# Use Bart to generate storyline from storytitle
References to 
https://towardsdatascience.com/teaching-bart-to-rap-fine-tuning-hugging-faces-bart-model-41749d38f3ef

In [1]:
from google.colab import drive
drive.mount('/content/drive')
root_dir = "/content/drive/MyDrive/CSCI544_Project/data"

Mounted at /content/drive


In [2]:
# This run uses Pytorch Lightening to finetune the model
!pip install -q pytorch-lightning
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.6/715.6 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.2 MB/s eta 0:00:00


In [3]:
# imports
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np

import torch.nn.functional as F
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint

import math
import random
import re
import argparse

In [4]:
df = pd.read_csv(root_dir + '/train_title_line_story.csv')
df

,storytitle,storyline,story,titleLineConcat
0,David Drops the Weight,put try realized started weeks,David noticed he had put on a lot of weight re...,David Drops the Weight <EOT> put try realized ...
1,Frustration,tom angry wall tom regret,Tom had a very short temper.One day a guest ma...,Frustration <EOT> tom angry wall tom regret
2,Marcus Buys Khakis,business formal pair perfectly marcus,Marcus needed clothing for a business casual e...,Marcus Buys Khakis <EOT> business formal pair ...
3,Different Opinions,trailer needed much ways truck,Bobby thought Bill should buy a trailer and ha...,Different Opinions <EOT> trailer needed much w...
4,Overcoming shortcomings,pastor tried sing sundays delighted,John was a pastor with a very bad memory.He tr...,Overcoming shortcomings <EOT> pastor tried sin...
...,...,...,...,...
98156,First Date,nervous two walked two evening,Jill was nervous because it was her first date...,First Date <EOT> nervous two walked two evening
98157,bus,work broke took broke ended,George had to get to work.His car broke down.H...,bus <EOT> work broke took broke ended
98158,Extra Moolah,since make tampa paid wish,"Since my husband retired last year, we don't h...",Extra Moolah <EOT> since make tampa paid wish
98159,New ipad,thrilled opened turned could buying,Suzy was thrilled to be getting her first ipad...,New ipad <EOT> thrilled opened turned could bu...


In [5]:
train, valid, test = np.split(df, [int(.8*len(df)), int(.9*len(df))])

In [6]:
train

,storytitle,storyline,story,titleLineConcat
0,David Drops the Weight,put try realized started weeks,David noticed he had put on a lot of weight re...,David Drops the Weight <EOT> put try realized ...
1,Frustration,tom angry wall tom regret,Tom had a very short temper.One day a guest ma...,Frustration <EOT> tom angry wall tom regret
2,Marcus Buys Khakis,business formal pair perfectly marcus,Marcus needed clothing for a business casual e...,Marcus Buys Khakis <EOT> business formal pair ...
3,Different Opinions,trailer needed much ways truck,Bobby thought Bill should buy a trailer and ha...,Different Opinions <EOT> trailer needed much w...
4,Overcoming shortcomings,pastor tried sing sundays delighted,John was a pastor with a very bad memory.He tr...,Overcoming shortcomings <EOT> pastor tried sin...
...,...,...,...,...
78523,Bad Milk,hungry walked milk threw cup,"One morning, Sarah woke up hungry.She walked i...",Bad Milk <EOT> hungry walked milk threw cup
78524,Ned,bike ride work work rainy,Ned rode his bike to work everyday because he ...,Ned <EOT> bike ride work work rainy
78525,Alone,sleeping every would tonight told,Eric hated sleeping alone.So he would bring a ...,Alone <EOT> sleeping every would tonight told
78526,No Customer Support,credit tried unfortunately concerns phone,Kris called her credit card company.She tried ...,No Customer Support <EOT> credit tried unfortu...


In [7]:
valid

,storytitle,storyline,story,titleLineConcat
78528,Stomach,tom much stomach decided thin,Tom was overweight.He always ate too much.He f...,Stomach <EOT> tom much stomach decided thin
78529,Standardized test,take quite well fine sigh,Alicia had to take a standardized test.She was...,Standardized test <EOT> take quite well fine sigh
78530,Mitchell Learns a New Sport,mitchell wanted could every mitchell,Mitchell has always been good at basketball.He...,Mitchell Learns a New Sport <EOT> mitchell wan...
78531,Writing,ted write style regularly went,Ted was a student in the third grade.His teach...,Writing <EOT> ted write style regularly went
78532,Sprayed by a Skunk,pee back smells sprayed outside,Cody lets his dog out to pee.The dog comes bac...,Sprayed by a Skunk <EOT> pee back smells spray...
...,...,...,...,...
88339,Electrocuted,sam screwed shock electrocuted sam,Sam was changing the light bulb.He screwed in ...,Electrocuted <EOT> sam screwed shock electrocu...
88340,Cold Coffee,order made ordered waited disappointed,Janet went to the coffee shop to order her mor...,Cold Coffee <EOT> order made ordered waited di...
88341,Carlos Has Dandruff,proud unfortunately store gradually proud,Carlos has always been proud of his thick blac...,Carlos Has Dandruff <EOT> proud unfortunately ...
88342,A New Car,new unfortunately luckily ajay new,"Ajay wanted a new car.Unfortunately, he couldn...",A New Car <EOT> new unfortunately luckily ajay...


In [8]:
test

,storytitle,storyline,story,titleLineConcat
88344,Jack's Bullies,way take plan dumped scream,Jack was getting bullied on his way to school....,Jack's Bullies <EOT> way take plan dumped scream
88345,Jewelry,thomas starting plan part pick,Thomas had a problem where he couldn't stop bu...,Jewelry <EOT> thomas starting plan part pick
88346,Appendicitis,henry appendicitis started ultimately worry,Henry had to go in for surgery last week.He ha...,Appendicitis <EOT> henry appendicitis started ...
88347,Photo Op,wife disabled bridge took good,Last week I was out with my wife.I rarely go o...,Photo Op <EOT> wife disabled bridge took good
88348,Jersey,play skill told neon team,Melo really liked to play sports.His basketbal...,Jersey <EOT> play skill told neon team
...,...,...,...,...
98156,First Date,nervous two walked two evening,Jill was nervous because it was her first date...,First Date <EOT> nervous two walked two evening
98157,bus,work broke took broke ended,George had to get to work.His car broke down.H...,bus <EOT> work broke took broke ended
98158,Extra Moolah,since make tampa paid wish,"Since my husband retired last year, we don't h...",Extra Moolah <EOT> since make tampa paid wish
98159,New ipad,thrilled opened turned could buying,Suzy was thrilled to be getting her first ipad...,New ipad <EOT> thrilled opened turned could bu...


In [9]:
# Create a dataloading module as per the PyTorch Lightning Docs
SOURCE = 'storytitle'
TARGET = 'storyline'

class SummaryDataModule(pl.LightningDataModule):
  def __init__(self, tokenizer, train, valid, batch_size):
    super().__init__()
    self.tokenizer = tokenizer
    self.train = train
    self.validate = valid
    self.batch_size = batch_size
  
  # Loads and splits the data into training, validation and test sets with a 80/10/10 split
  # def prepare_data(self):
    # self.data = pd.read_csv(self.data_file)[:self.num_examples]
    # self.train, self.validate, self.test = np.split(self.data.sample(frac=1), [int(.8*len(self.data))])

  # encode the sentences using the tokenizer  
  def setup(self, stage):
    self.train = encode_sentences(self.tokenizer, self.train[SOURCE], self.train[TARGET])
    self.validate = encode_sentences(self.tokenizer, self.validate[SOURCE], self.validate[TARGET])
    # self.test = encode_sentences(self.tokenizer, self.test[SOURCE], self.test[TARGET])

  # Load the training, validation and test sets in Pytorch Dataset objects
  def train_dataloader(self):
    dataset = TensorDataset(self.train['input_ids'], self.train['attention_mask'], self.train['labels'])                          
    train_data = DataLoader(dataset, sampler = RandomSampler(dataset), batch_size = self.batch_size)
    return train_data

  def val_dataloader(self):
    dataset = TensorDataset(self.validate['input_ids'], self.validate['attention_mask'], self.validate['labels']) 
    val_data = DataLoader(dataset, batch_size = self.batch_size)                       
    return val_data

  # def test_dataloader(self):
  #   dataset = TensorDataset(self.test['input_ids'], self.test['attention_mask'], self.test['labels']) 
  #   test_data = DataLoader(dataset, batch_size = self.batch_size)                   
  #   return test_data



In [10]:
class LitModel(pl.LightningModule):
  # Instantiate the model
  def __init__(self, learning_rate, tokenizer, model, hparams):
    super().__init__()
    self.tokenizer = tokenizer
    self.model = model
    self.learning_rate = learning_rate
    # self.freeze_encoder = freeze_encoder
    # self.freeze_embeds_ = freeze_embeds
    # self.hparams = hparams
    self.save_hyperparameters(hparams)

    if self.hparams.freeze_encoder:
      freeze_params(self.model.get_encoder())

    if self.hparams.freeze_embeds:
      self.freeze_embeds()
  
  def freeze_embeds(self):
    ''' freeze the positional embedding parameters of the model; adapted from finetune.py '''
    freeze_params(self.model.model.shared)
    for d in [self.model.model.encoder, self.model.model.decoder]:
      freeze_params(d.embed_positions)
      freeze_params(d.embed_tokens)

  # Do a forward pass through the model
  def forward(self, input_ids, **kwargs):
    return self.model(input_ids, **kwargs)
  
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr = self.learning_rate)
    return optimizer

  def training_step(self, batch, batch_idx):
    # Load the data into variables
    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]
    # Shift the decoder tokens right (but NOT the tgt_ids)
    decoder_input_ids = shift_tokens_right(tgt_ids, tokenizer.pad_token_id)

    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]
    # Create the loss function
    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    # Calculate the loss on the un-shifted tokens
    loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))

    return {'loss':loss}

  def validation_step(self, batch, batch_idx):

    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]

    decoder_input_ids = shift_tokens_right(tgt_ids, tokenizer.pad_token_id)
    
    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]

    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    val_loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))

    # self.log('val_loss', val_loss)

    return {'loss': val_loss}
  
  # Method that generates text using the BartForConditionalGeneration's generate() method
  def generate_text(self, text, eval_beams, early_stopping = False, max_len = 5):
    ''' Function to generate text '''
    generated_ids = self.model.generate(
        text["input_ids"],
        attention_mask=text["attention_mask"],
        use_cache=True,
        decoder_start_token_id = self.tokenizer.pad_token_id,
        num_beams= eval_beams,
        max_length = max_len,
        early_stopping = early_stopping
    )
    return [self.tokenizer.decode(w, skip_special_tokens=True, clean_up_tokenization_spaces=True) for w in generated_ids]

    # def training_epoch_end(self,outputs):
    #     #  the function is called after every epoch is completed

    #     # calculating average loss  
    #     avg_loss = torch.stack([x['loss'] for x in outputs]).mean()

    #     # calculating correect and total predictions
    #     # correct=sum([x["correct"] for  x in outputs])
    #     # total=sum([x["total"] for  x in outputs])

    #     # creating log dictionary
    #     # tensorboard_logs = {'loss': avg_loss,"Accuracy": correct/total}

    #     # epoch_dictionary={
    #     #     # required
    #     #     'loss': avg_loss,
            
    #     #     # for logging purposes
    #     #     'log': tensorboard_logs}

    #     # return epoch_dictionary
    #     self.log('train_loss', avg_loss)



def freeze_params(model):
  ''' Function that takes a model as input (or part of a model) and freezes the layers for faster training
      adapted from finetune.py '''
  for layer in model.parameters():
    layer.requires_grade = False


In [11]:
# Create the hparams dictionary to pass in the model
# I realise that this isn't really how this is meant to be used, but having this here reminds me that I can edit it when I need
hparams = argparse.Namespace()

hparams.freeze_encoder = True
hparams.freeze_embeds = True
hparams.eval_beams = 4

In [12]:
def shift_tokens_right(input_ids, pad_token_id):
  """ Shift input ids one token to the right, and wrap the last non pad token (usually <eos>).
      This is taken directly from modeling_bart.py
  """
  prev_output_tokens = input_ids.clone()
  index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
  prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
  prev_output_tokens[:, 1:] = input_ids[:, :-1]
  return prev_output_tokens

def encode_sentences(tokenizer, source_sentences, target_sentences, max_length=32, pad_to_max_length=True, return_tensors="pt"):
  ''' Function that tokenizes a sentence 
      Args: tokenizer - the BART tokenizer; source and target sentences are the source and target sentences
      Returns: Dictionary with keys: input_ids, attention_mask, target_ids
  '''

  input_ids = []
  attention_masks = []
  target_ids = []
  tokenized_sentences = {}

  for sentence in source_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim = 0)
  attention_masks = torch.cat(attention_masks, dim = 0)

  for sentence in target_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )
    # Shift the target ids to the right
    # shifted_target_ids = shift_tokens_right(encoded_dict['input_ids'], tokenizer.pad_token_id)
    target_ids.append(encoded_dict['input_ids'])

  target_ids = torch.cat(target_ids, dim = 0)
  

  batch = {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
      "labels": target_ids,
  }

  return batch


# def noise_sentence(sentence_, percent_words, replacement_token = "<mask>"):
#   '''
#   Function that noises a sentence by adding <mask> tokens
#   Args: sentence - the sentence to noise
#         percent_words - the percent of words to replace with <mask> tokens; the number is rounded up using math.ceil
#   Returns a noised sentence
#   '''
#   # Create a list item and copy
#   sentence_ = sentence_.split(' ')
#   sentence = sentence_.copy()
  
#   num_words = math.ceil(len(sentence) * percent_words)
  
#   # Create an array of tokens to sample from; don't include the last word as an option because in the case of lyrics
#   # that word is often a rhyming word and plays an important role in song construction
#   sample_tokens = set(np.arange(0, np.maximum(1, len(sentence)-1)))
  
#   words_to_noise = random.sample(sample_tokens, num_words)
  
#   # Swap out words, but not full stops
#   for pos in words_to_noise:
#       if sentence[pos] != '.':
#           sentence[pos] = replacement_token
  
#   # Remove redundant spaces
#   sentence = re.sub(r' {2,5}', ' ', ' '.join(sentence))
  
#   # Combine concurrent <mask> tokens into a single token; this just does two rounds of this; more could be done
#   sentence = re.sub(r'<mask> <mask>', "<mask>", sentence)
#   sentence = re.sub(r'<mask> <mask>', "<mask>", sentence)
#   return sentence
  

# Load BART
Here we load the model. I used "bart-base" because I had memory issues using "bart-large". "bart-base" appears to load without the use_cache argument, which by necessity must be turned to "False" for "bart-large".

In [13]:
# Load the model
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base', add_prefix_space=True)

bart_model = BartForConditionalGeneration.from_pretrained(
    "facebook/bart-base")


In [18]:
# Load the data into the model for training
summary_data = SummaryDataModule(tokenizer, train, test, batch_size = 128)

# Load the model from a pre-saved checkpoint; alternatively use the code below to start training from scratch
# model = LitModel.load_from_checkpoint(root_dir + "checkpoint_files_2/10_epoch_11_29_lt.ckpt",
#                                       learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model, hparams = hparams)

model = LitModel(learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model, hparams = hparams)

# Training the model with Pytorch Lightning
The below code utilises Pytorch Lightning's fantastic Trainer module that helps to control the training process. After creating a ModelCheckpoint object, the other options are fed into the Trainer module. I found that my colab crashed when I didn't explicitly set progress_bar_refresh_rate to something and I found that setting it to 500 seemed to work just fine.

In [19]:

checkpoint = ModelCheckpoint(dirpath=root_dir + '/checkpoint_files_2/')
trainer = pl.Trainer(#gpus = 1,
                     max_epochs = 1,
                     min_epochs = 1)
                     #auto_lr_find = False,
                     #checkpoint_callback = checkpoint,
                     #progress_bar_refresh_rate = 500)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [20]:
# Fit the instantiated model to the data
trainer.fit(model, summary_data)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | BartForConditionalGeneration | 139 M 
-------------------------------------------------------
139 M     Trainable params
0         Non-trainable params
139 M     Total params
557.682   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [21]:
# %load_ext tensorboard
# %tensorboard --logdir /content/lightning_logs/

In [22]:
# If you want to manually save a checkpoint, this works, although the model should automatically save (progressively better)
# checkpoints as it moves through the epochs
trainer.save_checkpoint(root_dir + "/checkpoint_files_2/5_epoch_11_30_lt.ckpt")

# Generate Storylines

In [23]:
def generate_storyline(title, model_):
  # Put the model on eval mode
  model_.to(torch.device('cpu'))
  model_.eval()
  prompt_line_tokens = tokenizer(title, max_length = 100, return_tensors = "pt", truncation = True)
  out = model.generate_text(prompt_line_tokens, eval_beams = 4, early_stopping = True, max_len=1000)
  return out[0].strip()

In [24]:
storytitle = df['storytitle'][0]
storytitle

'David Drops the Weight'

In [25]:
storyline = df['storyline'][0]
storyline

'put try realized started weeks'

In [26]:
storyline = generate_storyline(storytitle, model)
storyline
#3 epoch
#Frustration
#jimmy told told told said
#David Drops the Weight
#dave overweight weight weight weight
#8 epoches
#David Drops the Weight
#lose wanted diet doctor doctor
#Frustration
#jim jim told told jim
#13 epoches
# Frustration
# tom angry wall tom tom
# David Drops the Weight
# pounds decided went month pounds

'dave diet diet diet weight'

In [27]:
test

,storytitle,storyline,story,titleLineConcat
88344,Jack's Bullies,way take plan dumped scream,Jack was getting bullied on his way to school....,Jack's Bullies <EOT> way take plan dumped scream
88345,Jewelry,thomas starting plan part pick,Thomas had a problem where he couldn't stop bu...,Jewelry <EOT> thomas starting plan part pick
88346,Appendicitis,henry appendicitis started ultimately worry,Henry had to go in for surgery last week.He ha...,Appendicitis <EOT> henry appendicitis started ...
88347,Photo Op,wife disabled bridge took good,Last week I was out with my wife.I rarely go o...,Photo Op <EOT> wife disabled bridge took good
88348,Jersey,play skill told neon team,Melo really liked to play sports.His basketbal...,Jersey <EOT> play skill told neon team
...,...,...,...,...
98156,First Date,nervous two walked two evening,Jill was nervous because it was her first date...,First Date <EOT> nervous two walked two evening
98157,bus,work broke took broke ended,George had to get to work.His car broke down.H...,bus <EOT> work broke took broke ended
98158,Extra Moolah,since make tampa paid wish,"Since my husband retired last year, we don't h...",Extra Moolah <EOT> since make tampa paid wish
98159,New ipad,thrilled opened turned could buying,Suzy was thrilled to be getting her first ipad...,New ipad <EOT> thrilled opened turned could bu...


In [28]:
reduced_test, _= np.split(test, [int(.05*len(test))])
reduced_test

,storytitle,storyline,story,titleLineConcat
88344,Jack's Bullies,way take plan dumped scream,Jack was getting bullied on his way to school....,Jack's Bullies <EOT> way take plan dumped scream
88345,Jewelry,thomas starting plan part pick,Thomas had a problem where he couldn't stop bu...,Jewelry <EOT> thomas starting plan part pick
88346,Appendicitis,henry appendicitis started ultimately worry,Henry had to go in for surgery last week.He ha...,Appendicitis <EOT> henry appendicitis started ...
88347,Photo Op,wife disabled bridge took good,Last week I was out with my wife.I rarely go o...,Photo Op <EOT> wife disabled bridge took good
88348,Jersey,play skill told neon team,Melo really liked to play sports.His basketbal...,Jersey <EOT> play skill told neon team
...,...,...,...,...
88829,Too cold to sleep.,maggie much warm use turn,"Maggie was lying in bed, freezing cold.Her hus...",Too cold to sleep. <EOT> maggie much warm use ...
88830,Game show winning,gameshow would show car went,Alexia and her friends went on a gameshow.They...,Game show winning <EOT> gameshow would show ca...
88831,Finger,holidays year saving piece using,Connie loves making crafts for the holidays.Th...,Finger <EOT> holidays year saving piece using
88832,Changing Room,school gym nervous sat entered,Gina entered the locker room at school.Everyon...,Changing Room <EOT> school gym nervous sat ent...


In [29]:
remove_index = []
generated_lines = []
for index, row in reduced_test.iterrows():
    out = generate_storyline(row['storytitle'], model)
    if len(out.split()) != 5 or '.' in out.split():
      remove_index.append(index)
      print(out)
    else:
      generated_lines.append(out)

hamster hamster tried hamster
hamster hamster went hamster
potty potty use potty
prom prom prom date prom prom
jimmy throw throw throw threw throw
toaster toaster went toaster fire toaster
tony tomato sauce sauce sauce tony
band-Aid went band-aid went
jim puzzle piece puzzle puzzle puzzle
travis travis go travis skiing travis
shell shell shells shell shell shell
tess drinking tess tess
jane jelly beans jelly beans jane


In [30]:
reduced_test.drop(remove_index, axis=0, inplace=True)

In [31]:
reduced_test['predicted_storylines'] = generated_lines

In [32]:
for index, row in reduced_test.iterrows():
    out = row['predicted_storylines']
    if len(out.split()) != 5 or '.' in out.split():
      print(out)

In [33]:
reduced_test

,storytitle,storyline,story,titleLineConcat,predicted_storylines
88344,Jack's Bullies,way take plan dumped scream,Jack was getting bullied on his way to school....,Jack's Bullies <EOT> way take plan dumped scream,japan school japan bully japan
88345,Jewelry,thomas starting plan part pick,Thomas had a problem where he couldn't stop bu...,Jewelry <EOT> thomas starting plan part pick,jewelry went found found found
88346,Appendicitis,henry appendicitis started ultimately worry,Henry had to go in for surgery last week.He ha...,Appendicitis <EOT> henry appendicitis started ...,jane jane went went jane
88347,Photo Op,wife disabled bridge took good,Last week I was out with my wife.I rarely go o...,Photo Op <EOT> wife disabled bridge took good,jim jim photo photo jim
88348,Jersey,play skill told neon team,Melo really liked to play sports.His basketbal...,Jersey <EOT> play skill told neon team,jimmy wanted went went went
...,...,...,...,...,...
88829,Too cold to sleep.,maggie much warm use turn,"Maggie was lying in bed, freezing cold.Her hus...",Too cold to sleep. <EOT> maggie much warm use ...,jimmy tried tried tried sleep
88830,Game show winning,gameshow would show car went,Alexia and her friends went on a gameshow.They...,Game show winning <EOT> gameshow would show ca...,jimmy show show show winning
88831,Finger,holidays year saving piece using,Connie loves making crafts for the holidays.Th...,Finger <EOT> holidays year saving piece using,finger went finger finger finger
88832,Changing Room,school gym nervous sat entered,Gina entered the locker room at school.Everyon...,Changing Room <EOT> school gym nervous sat ent...,room room room roommate roommate


In [34]:
def concat_title_line(x):
    l = x['storytitle'].split()
    l.append('<EOT>')
    l.extend(x['predicted_storylines'].split())
    return ' '.join(l)

reduced_test['predicted_titleLineConcat'] = reduced_test.apply(lambda x: concat_title_line(x), axis=1)

In [35]:
reduced_test

,storytitle,storyline,story,titleLineConcat,predicted_storylines,predicted_titleLineConcat
88344,Jack's Bullies,way take plan dumped scream,Jack was getting bullied on his way to school....,Jack's Bullies <EOT> way take plan dumped scream,japan school japan bully japan,Jack's Bullies <EOT> japan school japan bully ...
88345,Jewelry,thomas starting plan part pick,Thomas had a problem where he couldn't stop bu...,Jewelry <EOT> thomas starting plan part pick,jewelry went found found found,Jewelry <EOT> jewelry went found found found
88346,Appendicitis,henry appendicitis started ultimately worry,Henry had to go in for surgery last week.He ha...,Appendicitis <EOT> henry appendicitis started ...,jane jane went went jane,Appendicitis <EOT> jane jane went went jane
88347,Photo Op,wife disabled bridge took good,Last week I was out with my wife.I rarely go o...,Photo Op <EOT> wife disabled bridge took good,jim jim photo photo jim,Photo Op <EOT> jim jim photo photo jim
88348,Jersey,play skill told neon team,Melo really liked to play sports.His basketbal...,Jersey <EOT> play skill told neon team,jimmy wanted went went went,Jersey <EOT> jimmy wanted went went went
...,...,...,...,...,...,...
88829,Too cold to sleep.,maggie much warm use turn,"Maggie was lying in bed, freezing cold.Her hus...",Too cold to sleep. <EOT> maggie much warm use ...,jimmy tried tried tried sleep,Too cold to sleep. <EOT> jimmy tried tried tri...
88830,Game show winning,gameshow would show car went,Alexia and her friends went on a gameshow.They...,Game show winning <EOT> gameshow would show ca...,jimmy show show show winning,Game show winning <EOT> jimmy show show show w...
88831,Finger,holidays year saving piece using,Connie loves making crafts for the holidays.Th...,Finger <EOT> holidays year saving piece using,finger went finger finger finger,Finger <EOT> finger went finger finger finger
88832,Changing Room,school gym nervous sat entered,Gina entered the locker room at school.Everyon...,Changing Room <EOT> school gym nervous sat ent...,room room room roommate roommate,Changing Room <EOT> room room room roommate ro...


In [36]:
reduced_test.to_csv(root_dir+'/test_predicted_storyline_5_ep.csv', encoding='utf-8', index=False)

In [ ]:
#output notebook as pdf
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('bart_title_2_storyline.ipynb')